In [1]:
import pandas as pd

## Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [38]:
ratings = pd.read_csv(r'c:\Users\petrenkovn\Dropbox\DS-22\Data_Files\ml-latest-small\ratings.csv')

# подготовка датафрейма со списком пользователей, у которых более 100 оценок
ratings_by_user = ratings[['userId','movieId']].groupby(['userId']).count().reset_index()
users_with_more_100 = ratings_by_user[ ratings_by_user['movieId'] > 100 ][['userId']]

# фильтрация исходного датафрейма и подготовка датафреймов с максимальной и минимальной меткой времени в разрезе пользователя
ratings_filtered = ratings.merge(users_with_more_100, how='inner', on='userId')

users_maxtimestamp = ratings_filtered[['userId','timestamp']].groupby(['userId']).max()
users_maxtimestamp.rename(columns={'timestamp': 'max'}, inplace=True)

users_mintimestamp = ratings_filtered[['userId','timestamp']].groupby(['userId']).min()
users_mintimestamp.rename(columns={'timestamp': 'min'}, inplace=True)

# присоединение двух последних датафреймов к первому и создание вычисляемого столбца
users_with_more_100 = users_with_more_100.merge(users_maxtimestamp, on='userId')
users_with_more_100 = users_with_more_100.join(users_mintimestamp, on='userId')
users_with_more_100['lifetime'] = ( users_with_more_100['max'] - users_with_more_100['min'] ) / 86400
lifetime = users_with_more_100['lifetime'].mean()
print(f'Время жизни: {int(lifetime)} дней')

Время жизни: 517 дней


## Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [22]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

# первая таблица
data = pd.concat([rzd,auto,air]).groupby(['client_id']).sum().reset_index()
# вторая таблица
data = data.merge(client_base, on='client_id', how='outer')
data

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,57483.0,0.0,Левобережная 1а
3,114,5774.0,83.0,0.0,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,0.0,4834.0,4.0,Строителей 18
6,117,0.0,98.0,13.0,Панфиловская 33
7,118,0.0,0.0,173.0,Мастеркова 4


## Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

1. Для связывания время-зависимых координат пользователя с фактами визитов/покупок, думаю нужно воспользоваться концепцией Slowly Changing Dimensions Type 2. В ее основе лежит идея соотненесия каждого значения признака некоторого ключа с временнЫм интервалом, в течение которого действовал это значение признака. Как именно применить концепцию, нужно смотреть применительно к данным. Как вариант, если в таблице с координатами есть детализация до даты-времени, можно на ее основе подготовить таблицу с детализацией "Пользователь-Дата-Час-Координаты" и ее присоединять к таблицам фактов по первым трем ключам.

2. Значительная удаленность координат отразится на аналитике ожидаемым образом - факты транзакций данного пользователя будут привязаны к совершенно разным локациям. Например, если пользователь делал заказы, находясь в путешествии, то эти заказы будут отнесены к его местоположению на момент совершения покупки, и оно будет отличаться от "домашнего" местоположения. Другой вопрос, нужно ли учитывать этот фактор в аналитике. Но коль скоро поставлен вопрос о время-зависимости координат, учитывать нужно. 

3. Для получения дополнительных признаков из координат можно воспользоваться сервисами обратного гео-кодирования, переводящего координаты в строку с адресом. Например Геокодер Яндекс Карт https://yandex.ru/dev/maps/geocoder/. Из строки адреса можно вытащить географические признаки, такие как город. 